In [ ]:
import os # to check if files exist
import re #to split the text file
import random #to choose better
import requests #to send the cards to anki
from concurrent.futures import ThreadPoolExecutor #to speed up sending the cards to anki

# Read the content of the file
file_path = "test your understanding.txt" #TODO you might have to change this
with open(file_path, "r") as file:
    content = file.read()

# Split the content at "Test your understanding"
substrings = content.split("Test your understanding")

# Initialize a nested list to store non-empty lines after each substring
nested_list = []

#split before new task
pattern = re.compile(r'\n(?=.{1,2}\))')

# Process each substring and split non-empty lines
for substring in substrings[1:]:  # Start from index 1 to skip the first empty substring
    lines = re.split(pattern, substring)
    nested_list.append(lines[1:]) #first line is also empty

In [ ]:
#TODO If you put answers in a answer file, use the format "<tut number> <question id (e.g. 'j' or '10')>) <answer>"

# Read answer files and put them in the answer dict
answer_dict = {}

def split_tut_question_text(string):
        return re.findall(pattern= r'^(\d{1,2})\s(.{1,2}\))\s(.+)', string = string)

def append_to_answer_dict(lines_answers, name):
        for line in lines_answers:
            answer = split_tut_question_text(string = line)
            tut_num = int(answer[0][0]) 
            question_num = answer[0][1]
            answer_text = f"{name}- " + answer[0][2]
            if tut_num not in answer_dict:
                answer_dict[tut_num] = {question_num:answer_text}
            else: 
                if question_num not in answer_dict[tut_num]:
                    answer_dict[tut_num][question_num] = answer_text
                else:
                    answer_dict[tut_num][question_num] = answer_dict[tut_num][question_num] + "\n" + answer_text

answer_files = []
for file in os.listdir():
    match = re.match(r"test your understanding answers (.*)\.txt", file)
    if match:
        file_path = os.path.join(os.getcwd(), file)
        with open(file_path, "r") as file_content:
            answer_files.append([file, match.group(1), file_content.read()])

for file in answer_files:
    answers_unsplit = file[2]
    name = file[1]
    
    pattern_answer_lines = re.compile(r'\n(?=\d{1,2} .{1,2}\))')
    lines_answers = re.split(pattern_answer_lines, answers_unsplit)
    
    append_to_answer_dict(lines_answers, name = name)

#function to get question num
def get_question_num(question):
    return  re.findall(pattern = r"^.{1,2}\)", string = question)[0]

#function to find answer
def find_answer(question, tut_num):
    question_num = get_question_num(question)
    answer = "Didn't answer yet"
    if tut_num in answer_dict.keys():
        if question_num in answer_dict[tut_num].keys():
            answer = answer_dict[tut_num][question_num]
    return(answer)


#function test yoursef using a random subset of numbers
def ask_question(question):
    return input(question + '\n')

def test(n = 1, show_questions_first = False, inclue_answers = True, ask_answers = True):
    """
    n: number of questions to be asked
    show_questions_first: print all drawn questions
    inclue_answers: include answers when printing drawn questions
    ask_answers: ask answers to questions. Inputs will be included in answer dict (which will be included in anki cards)
    """
     
    questions = []
    for i in range(n):
        #get random tut
        random_tut = random.choice(nested_list)
        # get tut num
        tut_num = nested_list.index(random_tut) +1
        # choose a random question from the tut
        question = random.choice(random_tut)

        if show_questions_first:
            print(f"Tut: {tut_num}\n{question}")
            if inclue_answers:
                print(f"Answer: {find_answer(question, tut_num)}")
        questions.append(f"{tut_num} {question}")
           
    if ask_answers:
        name = input("Under which name do you want to answer the questions?" + "\n")
        # List to store answers
        answers = []

        # Ask each question and store the answers
        for question in questions:
            tut_question_text = split_tut_question_text(question)
            answer = f"{tut_question_text[0][0]} {tut_question_text[0][1]} {ask_question(question)}"
            print(answer)
            answers.append(answer)


        # Convert the list of answers to lines in a .txt document
        filename = f"test your understanding answers {name}.txt"

        # Check if the file exists
        if os.path.exists(filename):
            # Append content to an existing file
            with open(filename, "a") as file:
                for answer in answers:
                    file.write(f"{answer}\n")
        else:
            # Create a new file
            with open(filename, "w") as file:
                for answer in answers:
                    file.write(f"{answer}\n")

        #put the answers into the answer_dict            
        append_to_answer_dict(answers, name)


In [ ]:
test(n = 2) #ask questions

In [ ]:
#TODO make shure Anki is installed and currently running on the machine
#TODO make shure anki-connect is installed https://github.com/FooSoft/anki-connect
#TODO make shure you have created an deck with the appropriate deck name 
#answer_dict = {} #TODO if you don' want to include answers, just uncomment this line 

#put all in Anki list without Answer
anki_list = [{"note_type": "Basic",
               "fields": {"Front":f"Tut {tut_i+1}\n{q}",
                          "Back": find_answer(q, tut_i+1)},
                "tags": [f"Tut_{tut_i+1}"]} for tut_i, tut in enumerate(nested_list) for q in tut]

def create_anki_card(payload):
    anki_url = "http://localhost:8765"  # AnkiConnect default URL
    response = requests.post(anki_url, json=payload)
    return response.json()

def create_anki_cards(cards_data):
    deck_name = "ML 2023 Test your understanding"  #TODO Change this to your deck name
    with ThreadPoolExecutor() as executor:
        payloads = []
        for card in cards_data:
            tags = ["ML", "Data_Science"]
            note_type = card["note_type"]
            fields = card["fields"]
            tags.extend(card["tags"])

            # Construct the payload for creating a new note
            payload = {
                "action": "addNote",
                "version": 6,
                "params": {
                    "note": {
                        "deckName": deck_name,
                        "modelName": note_type,
                        "fields": fields,
                        "tags": tags,  # Add tags if needed
                    }
                }
            }
            payloads.append(payload)
        # Use ThreadPoolExecutor to parallelize the requests
        responses = list(executor.map(create_anki_card, payloads))

        # Print the responses (optional)
        for response in responses:
            print(response)

create_anki_cards(anki_list) #Took half a minute on my machine